In [117]:
import sys
print(sys.executable)
# Verify kernal path
import numpy as np
import pandas as pd
import os
import requests
from dotenv import load_dotenv
load_dotenv()


prices = pd.read_parquet("../data/transformed/validated_prices_2022-08-02-2024-08-02.parquet") # reads in parquet-file in specified path and returns its data in a dataframe
prices

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime
0,56696985.0,164.7432,163.21,165.35,165.850,163.00,491310,2022-08-05 04:00:00
1,60362338.0,165.8939,166.37,164.87,167.810,164.20,540017,2022-08-08 04:00:00
2,63075503.0,164.8395,164.02,164.92,165.820,163.25,480552,2022-08-09 04:00:00
3,70170540.0,168.3496,167.68,169.24,169.340,166.90,559789,2022-08-10 04:00:00
4,57142109.0,169.3737,170.06,168.49,170.990,168.19,507914,2022-08-11 04:00:00
...,...,...,...,...,...,...,...,...
495,39827645.0,217.8992,218.70,217.96,219.490,216.01,608504,2024-07-26 04:00:00
496,35153729.0,218.1319,216.96,218.24,219.300,215.75,604680,2024-07-29 04:00:00
497,40681625.0,218.4059,219.19,218.80,220.325,216.12,584305,2024-07-30 04:00:00
498,48422974.0,222.3441,221.44,222.08,223.820,220.63,668833,2024-07-31 04:00:00


In [118]:
# HANDLE MISSING DAYS IN TIME-SERIES DATA: INTERPOLATION

start_date ="2022-08-02 04:00:00"
end_date = "2024-08-02 04:00:00"

start = pd.to_datetime("2022-08-02 04:00:00")
end = pd.to_datetime("2024-08-02 04:00:00")

prices['datetime'] = pd.to_datetime(prices['datetime'])

# creates column starting from start-date to end-date and including every day in between
full_date_range = pd.date_range(start=start_date, end=end_date, freq='D') 


existing_dates = prices["datetime"]
missing_dates = []

for date in full_date_range:
    if date not in existing_dates:
        missing_dates.append({
            'datetime': date,
            'open_price': np.nan,
            'high_price': np.nan,
            'low_price': np.nan,
            'close_price': np.nan,
            'volume': np.nan,
            'vw_avr_price': np.nan,
            'num_transactions': np.nan
        })

missing_rows_df = pd.DataFrame(missing_dates)
prices = pd.concat([prices, missing_rows_df]).drop_duplicates(subset=['datetime']).sort_values(by='datetime').reset_index(drop=True)

prices_interpolated = prices.interpolate(method='linear')
prices_interpolated.ffill(inplace=True)
prices_interpolated.bfill(inplace=True)


In [119]:
prices  # before interpolation, backfill, frontfill

,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-02 04:00:00
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-03 04:00:00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-04 04:00:00
3,56696985.0,164.7432,163.21,165.35,165.850,163.00,491310.0,2022-08-05 04:00:00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-06 04:00:00
...,...,...,...,...,...,...,...,...
897,35153729.0,218.1319,216.96,218.24,219.300,215.75,604680.0,2024-07-29 04:00:00
898,40681625.0,218.4059,219.19,218.80,220.325,216.12,584305.0,2024-07-30 04:00:00
899,48422974.0,222.3441,221.44,222.08,223.820,220.63,668833.0,2024-07-31 04:00:00
900,61125243.0,219.4773,224.37,218.36,224.480,217.02,876046.0,2024-08-01 04:00:00


In [120]:
prices_interpolated  # after interpolation, backfill, frontfill

,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime
0,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-02 04:00:00
1,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-03 04:00:00
2,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-04 04:00:00
3,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-05 04:00:00
4,5.791877e+07,165.126767,164.263333,165.19,166.503333,163.40,507545.666667,2022-08-06 04:00:00
...,...,...,...,...,...,...,...,...
897,3.515373e+07,218.131900,216.960000,218.24,219.300000,215.75,604680.000000,2024-07-29 04:00:00
898,4.068162e+07,218.405900,219.190000,218.80,220.325000,216.12,584305.000000,2024-07-30 04:00:00
899,4.842297e+07,222.344100,221.440000,222.08,223.820000,220.63,668833.000000,2024-07-31 04:00:00
900,6.112524e+07,219.477300,224.370000,218.36,224.480000,217.02,876046.000000,2024-08-01 04:00:00
